### Demo Preparation Notebook

**Please Note**: This notebook and demo are NOT intended to be used as learning materials. To gain
a thorough understanding of the DataJoint workflow for calcium imaging, please
see the [`tutorial`](./tutorial.ipynb) notebook.

In [ ]:
import datajoint as dj
import datetime
from workflow_array_ephys.pipeline import subject, session, probe, ephys
from element_array_ephys import ephys_report


In [ ]:
subject.Subject.insert1(
    dict(
        subject='subject6',
        subject_birth_date='2023-01-01',
        sex='U'
    )
)

In [ ]:
session_key = dict(subject='subject6', 
                   session_datetime=datetime.datetime.now())

session.Session.insert1(session_key)

session.SessionDirectory.insert1(
    dict(
        session_key, 
        session_dir='subject6/session1'
    )
)

In [ ]:
probe.Probe.insert1(
    dict(probe="17131311651", 
         probe_type="neuropixels 1.0 - 3B")
)

ephys.ProbeInsertion.insert1(
    dict(
        session_key,
        insertion_number=0,
        probe="17131311651",
    )
)

In [ ]:
populate_settings = {"display_progress": True}

ephys.EphysRecording.populate(**populate_settings)

In [ ]:
kilosort_params = {
    "fs": 30000,
    "fshigh": 150,
    "minfr_goodchannels": 0.1,
    "Th": [10, 4],
    "lam": 10,
    "AUCsplit": 0.9,
    "minFR": 0.02,
    "momentum": [20, 400],
    "sigmaMask": 30,
    "ThPr": 8,
    "spkTh": -6,
    "reorder": 1,
    "nskip": 25,
    "GPU": 1,
    "Nfilt": 1024,
    "nfilt_factor": 4,
    "ntbuff": 64,
    "whiteningRange": 32,
    "nSkipCov": 25,
    "scaleproc": 200,
    "nPCs": 3,
    "useRAM": 0,
}

ephys.ClusteringParamSet.insert_new_params(
    clustering_method="kilosort2",
    paramset_idx=0,
    params=kilosort_params,
    paramset_desc="Spike sorting using Kilosort2",
)

In [ ]:
ephys.ClusteringTask.insert1(
    dict(
        session_key,
        insertion_number=0,
        paramset_idx=0,
        task_mode='load', # load or trigger
        clustering_output_dir="subject6/session1/towersTask_g0_imec0",
    )
)

In [ ]:
ephys.Clustering.populate(**populate_settings)
ephys.CuratedClustering.populate(**populate_settings)
ephys.LFP.populate(**populate_settings)
ephys.WaveformSet.populate(**populate_settings)
ephys_report.ProbeLevelReport.populate(**populate_settings)
ephys_report.UnitLevelReport.populate(**populate_settings)